## Setup & Imports

In [24]:
import pandas as pd
import boto3
import json
from io import StringIO

# Configure S3
s3_client = boto3.client('s3', region_name='us-east-1')
BUCKET_NAME = 'akshays3-2026'

print("Setup complete")

Setup complete


/Users/akshay/Desktop/GitHub/rearcinfra/.venv/lib/python3.9/site-packages/boto3/compat.py:89: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


## Dataset Size Check (Pandas vs Spark)

Before choosing a compute engine, check the approximate dataset sizes we load in Part 3.

- **Pandas** runs on a single machine (single node) and typically keeps data in-memory. If the dataset fits comfortably in RAM, Pandas is often simplest and can be very fast.
- **Spark** is designed for distributed processing. It partitions data across multiple workers (a cluster) to process very large datasets in parallel, but has extra overhead (cluster + shuffle + serialization).

For this notebook, we use Pandas because the Part 3 inputs are typically small enough to fit on one node. The check below prints the input sizes so you can justify that choice.

In [ ]:
# Check sizes of the specific inputs used in Part 3
bls_key = 'raw/bls/pr.data.0.Current'
pop_key = 'raw/datausa/population.json'

def _mb(num_bytes: int) -> float:
    return num_bytes / (1024 * 1024)

def _gb(num_bytes: int) -> float:
    return num_bytes / (1024 * 1024 * 1024)

bls_head = s3_client.head_object(Bucket=BUCKET_NAME, Key=bls_key)
pop_head = s3_client.head_object(Bucket=BUCKET_NAME, Key=pop_key)

bls_bytes = int(bls_head['ContentLength'])
pop_bytes = int(pop_head['ContentLength'])
total_bytes = bls_bytes + pop_bytes

print(f"BLS file: s3://{BUCKET_NAME}/{bls_key}  ~{_mb(bls_bytes):.2f} MB")
print(f"POP file: s3://{BUCKET_NAME}/{pop_key}  ~{_mb(pop_bytes):.2f} MB")
print(f"Total input size (files on S3): ~{_mb(total_bytes):.2f} MB (~{_gb(total_bytes):.3f} GB)")

# Guidance (heuristic, not a strict threshold):
# - Pandas is often faster/simpler for small-to-medium data that fits in RAM.
# - Spark helps when data does not fit in RAM on one node, or when you need cluster-scale parallelism.
try:
    import psutil  # optional
    ram_bytes = int(psutil.virtual_memory().total)
    print(f"Detected RAM: ~{_gb(ram_bytes):.1f} GB")
    # Very rough safety margin: keep raw input well below total RAM (dataframes + overhead expand memory).
    if total_bytes < 0.25 * ram_bytes:
        print("Recommendation: Pandas is a good fit (inputs comfortably under RAM).")
    else:
        print("Recommendation: Consider Spark if you expect memory pressure on one node.")
except Exception:
    print("Recommendation: Use Pandas if these inputs fit comfortably in your machine's RAM; consider Spark for very large datasets or cluster-scale parallelism.")

BLS file: s3://akshays3-2026/raw/bls/pr.data.0.Current  ~1.49 MB
POP file: s3://akshays3-2026/raw/datausa/population.json  ~0.00 MB
Total input size (for notebook): ~1.49 MB
Recommendation: Pandas (single-node) is sufficient for these inputs.


## Load Data from S3

In [26]:
# Load BLS data
bls_key = 'raw/bls/pr.data.0.Current'
obj = s3_client.get_object(Bucket=BUCKET_NAME, Key=bls_key)
bls_content = obj['Body'].read().decode('utf-8')

bls_df = pd.read_csv(StringIO(bls_content), sep='\t', dtype=str)
bls_df.columns = bls_df.columns.str.strip()
bls_df = bls_df.map(lambda x: x.strip() if isinstance(x, str) else x)
bls_df['value'] = pd.to_numeric(bls_df['value'], errors='coerce')
bls_df['year'] = pd.to_numeric(bls_df['year'], errors='coerce')

print(f"BLS data shape: {bls_df.shape}")
bls_df.head()

BLS data shape: (37239, 5)


,series_id,year,period,value,footnote_codes
0,PRS30006011,1995,Q01,2.6,NaN
1,PRS30006011,1995,Q02,2.1,NaN
2,PRS30006011,1995,Q03,0.9,NaN
3,PRS30006011,1995,Q04,0.1,NaN
4,PRS30006011,1995,Q05,1.4,NaN


In [27]:
# Load population data
pop_key = 'raw/datausa/population.json'
obj = s3_client.get_object(Bucket=BUCKET_NAME, Key=pop_key)
pop_content = obj['Body'].read().decode('utf-8')
pop_json = json.loads(pop_content)

pop_rows = pop_json['data']  # DataUSA response stores records as a list under 'data'
pop_df = pd.DataFrame(pop_rows)
pop_df['Year'] = pd.to_numeric(pop_df['Year'], errors='coerce')
pop_df['Population'] = pd.to_numeric(pop_df['Population'], errors='coerce')

print(f"Population data shape: {pop_df.shape}")
pop_df.head()

Population data shape: (10, 4)


,Nation ID,Nation,Year,Population
0,01000US,United States,2013,316128839.0
1,01000US,United States,2014,318857056.0
2,01000US,United States,2015,321418821.0
3,01000US,United States,2016,323127515.0
4,01000US,United States,2017,325719178.0


## Query 1: Population Statistics (2013-2018)

Calculate mean and standard deviation of annual US population for years 2013-2018 inclusive.

In [28]:
# Filter years 2013-2018
pop_filtered = pop_df[(pop_df['Year'] >= 2013) & (pop_df['Year'] <= 2018)]

mean_pop = pop_filtered['Population'].mean()
std_pop = pop_filtered['Population'].std()

print(f"Mean Population (2013-2018): {mean_pop:,.0f}")
print(f"Standard Deviation: {std_pop:,.2f}")

pop_filtered[['Year', 'Population']]

Mean Population (2013-2018): 322,069,808
Standard Deviation: 4,158,441.04


,Year,Population
0,2013,316128839.0
1,2014,318857056.0
2,2015,321418821.0
3,2016,323127515.0
4,2017,325719178.0
5,2018,327167439.0


## Query 2: Best Year per Series

For every series_id, find the year with the maximum sum of quarterly values.

In [29]:
# Filter quarterly periods
quarterly = bls_df[bls_df['period'].str.startswith('Q', na=False)].copy()

# Group by series_id and year, sum values
grouped = quarterly.groupby(['series_id', 'year'])['value'].sum().reset_index()

# Find max year for each series
best_years = grouped.loc[grouped.groupby('series_id')['value'].idxmax()]
best_years = best_years.sort_values('series_id')

print(f"Total series analyzed: {len(best_years)}")
best_years.head(20)

Total series analyzed: 282


,series_id,year,value
27,PRS30006011,2022,20.500
58,PRS30006012,2022,17.100
65,PRS30006013,1998,705.895
108,PRS30006021,2010,17.700
139,PRS30006022,2010,12.400
174,PRS30006023,2014,503.216
213,PRS30006031,2022,20.500
243,PRS30006032,2021,17.100
251,PRS30006033,1998,702.672
306,PRS30006061,2022,37.000


## Query 3: Join Series with Population

For series_id = PRS30006032 and period = Q01, join with population data by year.

In [30]:
# Filter BLS data
filtered_bls = bls_df[
    (bls_df['series_id'] == 'PRS30006032') & 
    (bls_df['period'] == 'Q01')
].copy()

# Join with population
merged = filtered_bls.merge(
    pop_df,
    left_on='year',
    right_on='Year',
    how='left'
)

result = merged[['series_id', 'year', 'period', 'value', 'Population']]

print(f"Total matching records: {len(result)}")
result

Total matching records: 31


,series_id,year,period,value,Population
0,PRS30006032,1995,Q01,0.0,NaN
1,PRS30006032,1996,Q01,-4.2,NaN
2,PRS30006032,1997,Q01,2.8,NaN
3,PRS30006032,1998,Q01,0.9,NaN
4,PRS30006032,1999,Q01,-4.1,NaN
5,PRS30006032,2000,Q01,0.5,NaN
6,PRS30006032,2001,Q01,-6.3,NaN
7,PRS30006032,2002,Q01,-6.6,NaN
8,PRS30006032,2003,Q01,-5.7,NaN
9,PRS30006032,2004,Q01,2.0,NaN


## Summary

All three queries executed successfully:
- Query 1: Population statistics calculated
- Query 2: Best year per series identified
- Query 3: Series joined with population data